In [13]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data, page_req_key = quote_ctx.request_history_kline('HK.09988',ktype='K_DAY', start='2023-11-11', end='2023-11-16', max_count=5) # 5 per page, request the first page
if ret == RET_OK:
    min_close = data['close'].min() # The minimum closing price of the first page
    print(f"min_close: {min_close}")
else:
    print('error:', data)
quote_ctx.close() # After using the connection, remember to close it to prevent the number of connections from running out


2023-11-16 11:22:56,057 | 4107 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=13, host=127.0.0.1, port=11111, user_id=28749530
min_close: 79.2
2023-11-16 11:22:56,081 | 4107 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=13


In [10]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, data = quote_ctx.get_market_snapshot(['HK.09988'])
if ret == RET_OK:
    last_price = data['last_price'][0]  # 最新价格
    print(last_price)

else:
    print('can not get latest price', data)
quote_ctx.close() # 结束后记得关闭当条连接，防止连接条数用尽


2023-11-16 11:13:32,942 | 4107 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=10, host=127.0.0.1, port=11111, user_id=28749530
81.5
2023-11-16 11:13:32,954 | 4107 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=10


In [ ]:
from futu import *
trd_ctx = OpenSecTradeContext(filter_trdmarket=TrdMarket.HK, host='127.0.0.1', port=11111, security_firm=SecurityFirm.FUTUSECURITIES)
ret, history_data = trd_ctx.history_order_list_query()
if ret == RET_OK:
    if history_data.shape[0] > 0:  # If the order list is not empty
        # print order which status is filled 
        print(history_data[history_data['order_status'] == OrderStatus.FILLED_ALL])
else:
    print('history_order_list_query error: ', data)
trd_ctx.close()


In [10]:
from futu import *
trd_ctx = OpenSecTradeContext(filter_trdmarket=TrdMarket.HK, trd_env=TrdEnv.REAL,host='127.0.0.1', port=11111, security_firm=SecurityFirm.FUTUSECURITIES)
ret, data = trd_ctx.position_list_query()
if ret == RET_OK:
    print(data)
    if data.shape[0] > 0:  # If the position list is not empty
        print(data['stock_name'][0])  # Get the first stock name of the holding position
        print(data['stock_name'].values.tolist())  # Convert to list
else:
    print('position_list_query error: ', data)
trd_ctx.close()  # Close the current connection


Exception ignored in: <function OpenContextBase.__del__ at 0x12ec34900>
Traceback (most recent call last):
  File "/Users/whitetree/anaconda3/envs/futu_api/lib/python3.11/site-packages/futu/common/open_context_base.py", line 73, in __del__
    self.close()
  File "/Users/whitetree/anaconda3/envs/futu_api/lib/python3.11/site-packages/futu/trade/open_trade_context.py", line 32, in close
    super(OpenTradeContextBase, self).close()
  File "/Users/whitetree/anaconda3/envs/futu_api/lib/python3.11/site-packages/futu/common/open_context_base.py", line 116, in close
    self._close(False)
  File "/Users/whitetree/anaconda3/envs/futu_api/lib/python3.11/site-packages/futu/common/open_context_base.py", line 127, in _close
    with self._lock:
         ^^^^^^^^^^
AttributeError: 'OpenSecTradeContext' object has no attribute '_lock'


TypeError: OpenSecTradeContext.__init__() got an unexpected keyword argument 'trd_env'

In [ ]:
from futu import *
trd_ctx = OpenSecTradeContext(filter_trdmarket=TrdMarket.HK, host='127.0.0.1', port=11111, security_firm=SecurityFirm.FUTUSECURITIES)
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, data = trd_ctx.acctradinginfo_query(trd_env=TrdEnv.REAL, order_type=OrderType.NORMAL, code='HK.09988', price=88)
if ret == RET_OK:
    print(data)
    print(data['max_cash_and_margin_buy'][0])  # Get maximum quantity that can be bought on margin
else:
    print('acctradinginfo_query error: ', data)
trd_ctx.close()  # Close the current connection


In [ ]:
from futu import *
from utilts.encrypt import decrypt

PASSWORD = os.environ.get('FUTU_PASSWORD')
############################ 全局变量设置 ############################
FUTUOPEND_ADDRESS = '127.0.0.1'  # Futu OpenD 监听地址
FUTUOPEND_PORT = 11111  # Futu OpenD 监听端口

TRADING_ENVIRONMENT = TrdEnv.SIMULATE  # 交易环境：真实 / 模拟
TRADING_MARKET = TrdMarket.HK  # 交易市场权限，用于筛选对应交易市场权限的账户
TRADING_PWD = decrypt(PASSWORD)  # 交易密码，用于解锁交易
TRADING_PERIOD = KLType.K_1M  # 信号 K 线周期
TRADING_SECURITY = 'HK.09988'  # 交易标的

quote_context = OpenQuoteContext(host=FUTUOPEND_ADDRESS, port=FUTUOPEND_PORT)  # 行情对象
trade_context = OpenSecTradeContext(filter_trdmarket=TRADING_MARKET, host=FUTUOPEND_ADDRESS, port=FUTUOPEND_PORT, security_firm=SecurityFirm.FUTUSECURITIES)  # 交易对象，根据交易品种修改交易对象类型
# 解锁交易
def unlock_trade():
    if TRADING_ENVIRONMENT == TrdEnv.REAL:
        ret, data = trade_context.unlock_trade(TRADING_PWD)
        if ret != RET_OK:
            print('解锁交易失败：', data)
            return False
        print('解锁交易成功！')
    return True

############################ 填充以下函数来完成您的策略 ############################
# 策略启动时运行一次，用于初始化策略
def on_init():
    # 解锁交易（如果是模拟交易则不需要解锁）
    if not unlock_trade():
        return False
    print('************  策略开始运行 ***********')
    return True

on_init()